<a href="https://colab.research.google.com/github/bahram3000/test_balancer_app/blob/main/manage_positions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qqq web3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 93.5 MB/s eta 0:00:00


In [ ]:
from web3 import Web3
from eth_account import Account
import requests
import json
import time
import decimal
import numpy as np
import math

In [ ]:
from web3 import exceptions
from eth_abi import encode

In [ ]:
RPC = "https://eth-sepolia.g.alchemy.com/v2/1d3m81eR2Kd2mrpuH6-bR"
w3 = Web3(Web3.HTTPProvider(RPC))

# کلید خصوصی والت تست‌نت
PRIVATE_KEY = "75074ee5760d9640d993d09bc6e969b37cadcc2c83195886d43b3e86b2b8b6da"
account = Account.from_key(PRIVATE_KEY)
my_wallet_address = account.address

print("Connected:", w3.is_connected())
print("Your address:", my_wallet_address)

Connected: True
Your address: 0x7868EDEfC36A74eba220C75aCFB05c59a4e55822


In [ ]:
def fetch_abi(contract_address: str, api_key: str,inp_chain_id:int):
    url = "https://api.etherscan.io/v2/api"
    params = {
        "module": "contract",
        "action": "getabi",
        "address": contract_address,
        "apikey": api_key,
        "chainid":inp_chain_id
    }
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    if data.get("status") != "1":
        raise Exception(f"Error getting ABI: {data.get('message')}")
    abi = json.loads(data["result"])
    return abi

In [ ]:
etherscan_apikey = "N68HP1FGB4UQQFBHZQUJIM12HVYWJME96M"
CHAIN_ID = 11155111  # Sepolia

In [ ]:
position_manager_address = "0x1238536071E1c677A632429e3655c799b22cDA52"
position_manager_address = w3.to_checksum_address(position_manager_address)
position_manager_abi = fetch_abi(position_manager_address,etherscan_apikey,CHAIN_ID)
position_manager_contract = w3.eth.contract(address=position_manager_address, abi=position_manager_abi)

In [ ]:
def check_liquidity_v3_nft(inp_wallet_address,inp_postion_manger_contract):
  pm_contract=inp_postion_manger_contract
  address=inp_wallet_address
  nfts=[]
  try:
    balance = pm_contract.functions.balanceOf(address).call()
    print(f"تعداد NFT پوزیشن‌ها: {balance}")


    if balance > 0:
        for i in range(balance):
            time.sleep(1)
            token_id = pm_contract.functions.tokenOfOwnerByIndex(address, i).call()

            pos = pm_contract.functions.positions(token_id).call()
            dct={'token_id':token_id,'token0':pos[2],'token1':pos[3],'fee_tier':pos[4],'liquidity':pos[7],'tick_range':[pos[5],pos[6]],'fee0':pos[10],'fee1':pos[11]}
            nfts.append(dct)
        return nfts
    else:
        print("lp nft not found")

  except Exception as e:
    print(f"error in read data --> {e}")

In [ ]:
V3_LPS=check_liquidity_v3_nft(my_wallet_address,position_manager_contract)
V3_LPS[0]

تعداد NFT پوزیشن‌ها: 1


{'token_id': 221424,
 'token0': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',
 'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
 'fee_tier': 500,
 'liquidity': 33516997100150,
 'tick_range': [186440, 186640],
 'fee0': 0,
 'fee1': 0}

In [ ]:
v3factoryaddress="0x0227628f3F023bb0B980b67D528571c95c6DaC1c"
factory_address = w3.to_checksum_address(v3factoryaddress)
factory_abi =fetch_abi(factory_address,etherscan_apikey,CHAIN_ID)
factory=w3.eth.contract(address=factory_address, abi=factory_abi)

In [ ]:
def calculate_price_from_sqrt_price(sqrt_price_raw: float, decimal_0: int, decimal_1: int) -> float:
    normalized_sqrt_price = sqrt_price_raw / (2 ** 96)
    raw_price_math = normalized_sqrt_price ** 2
    decimal_adjustment = decimal_0 - decimal_1
    final_price = raw_price_math * (10 ** decimal_adjustment)
    return final_price

In [ ]:
def pool_current_info(inp_pool_contract):
  token0_address = inp_pool_contract.functions.token0().call()
  token1_address = inp_pool_contract.functions.token1().call()
  slot0 = inp_pool_contract.functions.slot0().call()
  liquidity = inp_pool_contract.functions.liquidity().call()
  fee = inp_pool_contract.functions.fee().call()
  tickspacing = inp_pool_contract.functions.tickSpacing().call()
  return {'token0_address':token0_address,'token1_address':token1_address,'sqrtprice':slot0[0],'current_tick':slot0[1],'liquidity':liquidity,'fee_tier':fee,'tick_spacing':tickspacing}

In [ ]:
def make_swap(input_token_address,output_token_address,amount_input:float,inp_wallet,inp_pool_address,swap_router_address,mode:str,inp_private_key):
  input_token_address=w3.to_checksum_address(input_token_address)
  erc20_abi=fetch_abi("0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14",etherscan_apikey,CHAIN_ID)
  input_token_contract=w3.eth.contract(address=input_token_address, abi=erc20_abi)
  input_token_contract=w3.eth.contract(address=input_token_address, abi=erc20_abi)
  output_token_address=w3.to_checksum_address(output_token_address)
  pool_address=w3.to_checksum_address(inp_pool_address)
  pool_abi=fetch_abi(pool_address,etherscan_apikey,CHAIN_ID)
  pool_contract=w3.eth.contract(address=pool_address, abi=pool_abi)
  swap_router_address=w3.to_checksum_address(swap_router_address)
  swap_router_abi=fetch_abi(swap_router_address,etherscan_apikey,CHAIN_ID)
  swap_router_contract=w3.eth.contract(address=swap_router_address, abi=swap_router_abi)
  pool_info=pool_current_info(pool_contract)
  amount_in_wei=amount_input*(10**input_token_contract.functions.decimals().call())
  if input_token_address in pool_info.values and output_token_address in pool_info.values:
    params = {
        "tokenIn": input_token_address,
        "tokenOut": output_token_address,
        "fee": pool_info['fee_tier'],
        "recipient": inp_wallet,
        "amountIn": amount_in_wei,
        "amountOutMinimum": 0,
        "sqrtPriceLimitX96": 0}
    nonce = w3.eth.get_transaction_count(inp_wallet)

    approve_tx = input_token_contract.functions.approve(
    swap_router_address,
    2*amount_in_wei).build_transaction({
    "from": inp_wallet,
    "nonce": w3.eth.get_transaction_count(inp_wallet),
    "gas": 100000,
    "gasPrice": w3.eth.gas_price})

    # 3. امضا و ارسال تراکنش approve
    signed_approve_tx = w3.eth.account.sign_transaction(approve_tx, inp_private_key)
    hash_approve = w3.eth.send_raw_transaction(signed_approve_tx.raw_transaction)
    print("Approve TX Sent:", w3.to_hex(hash_approve))

    swap_tx = swap_router_contract.functions.exactInputSingle(params).build_transaction({
      "from": inp_wallet,
      "gasPrice": w3.eth.gas_price})

    est_gas_need=w3.eth.estimate_gas(swap_tx)

    if mode=='send_raw':
      time.sleep(10)
      swap_tx = swap_router_contract.functions.exactInputSingle(params).build_transaction({
        "from": inp_wallet,
        "nonce": w3.eth.get_transaction_count(inp_wallet),
        "gas": est_gas_need,
        "gasPrice": w3.eth.gas_price})
      signed_tx = w3.eth.account.sign_transaction(swap_tx, inp_private_key)
      tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
      return w3.to_hex(tx_hash)

    elif mode=='simulate':
      return est_gas_need
    else:
      return False


In [ ]:
def check_token_balance(token_list_address:list, inp_address):
  ERC20_ABI = '''
  [
  {"constant":true,"inputs":[{"name":"_owner","type":"address"}],
   "name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],
   "type":"function"},
  {"constant":true,"inputs":[],"name":"decimals",
   "outputs":[{"name":"","type":"uint8"}],"type":"function"},
  {"constant":true,"inputs":[],"name":"symbol",
   "outputs":[{"name":"","type":"string"}],"type":"function"}
  ]
  '''
  balances = []
  for token in token_list_address:
      contract = w3.eth.contract(address=token, abi=ERC20_ABI)
      bal = contract.functions.balanceOf(inp_address).call()
      dec = contract.functions.decimals().call()
      sym = contract.functions.symbol().call()
      balances.append([sym, bal / (10 ** dec)])
  eth_balance = w3.eth.get_balance(inp_address)
  balances.append(["ETH", w3.from_wei(eth_balance, 'ether')])
  return balances

In [ ]:
def calculate_price_by_tick(tick:int,decimal_0:int,decimal_1:int):
  return (10**(decimal_0-decimal_1))*(1.0001**tick)

In [ ]:
def calculate_price_from_sqrt_price(sqrt_price_raw: float, decimal_0: int, decimal_1: int) -> float:
    normalized_sqrt_price = sqrt_price_raw / (2 ** 96)
    raw_price_math = normalized_sqrt_price ** 2
    decimal_adjustment = decimal_0 - decimal_1
    final_price = raw_price_math * (10 ** decimal_adjustment)
    return final_price

In [ ]:
def calculate_tick_by_price(price: float, decimal_0: int, decimal_1: int) -> int:
    if price <= 0:
        raise ValueError("Price must be greater than zero")
    decimal_factor = 10 ** (decimal_0 - decimal_1)
    val_to_log = math.log(price) - math.log(decimal_factor)
    tick_base = math.log(1.0001)
    tick = val_to_log / tick_base
    return int(round(tick))

In [ ]:
def calculate_required_amount_and_swap(
    current_price: float,
    price_lower: float,
    price_upper: float,
    amount0_wallet: float,
    amount1_wallet: float
) -> dict:
    if not (0 < price_lower < current_price < price_upper):
        return {
            "status": "Error",
            "message": "prices must be positive and current price should be between lower and upper"
        }

    sqrt_pc = math.sqrt(current_price)
    sqrt_pa = math.sqrt(price_lower)
    sqrt_pb = math.sqrt(price_upper)

    numerator = sqrt_pc - sqrt_pa

    denominator = (1 / sqrt_pc) - (1 / sqrt_pb)
    required_ratio = numerator / denominator
    v_total = (amount0_wallet * current_price) + amount1_wallet

    amount0_target = v_total / (current_price + required_ratio)

    amount1_target = amount0_target * required_ratio

    swap_diff_0 = amount0_wallet - amount0_target


    swap_action = ""
    swap_amount_abs = abs(swap_diff_0)

    if swap_diff_0 > 0:
        swap_action = f" {swap_amount_abs:.8f}sell token0 to buy token1 "
    elif swap_diff_0 < 0:
        swap_action = f"{swap_amount_abs:.8f}buy token0 by sell token1 "
    else:
        swap_action = "balance of tokens are ok"

    liquidity_denominator = (1 / sqrt_pc) - (1 / sqrt_pb)
    final_liquidity = amount0_target / liquidity_denominator
    #
    return {
        "status": "Success",
        "final_liquidity_L": final_liquidity,
        "ratio_required": required_ratio,
        "amount0_target": amount0_target,
        "amount1_target": amount1_target,
        "swap_details": swap_action,
        "swap_amount_token0": swap_diff_0
    }


In [ ]:
def calculate_new_amount_token_mix(current_price: float, lower: float, upper: float, liquidity: float):
    s = np.sqrt(current_price)
    su = np.sqrt(upper)
    sl = np.sqrt(lower)

    amount_0 = 0.0
    amount_1 = 0.0

    if current_price < lower:
        amount_0 = liquidity * ((1.0 / sl) - (1.0 / su))
        amount_1 = 0.0
    elif current_price > upper:
        amount_0 = 0.0
        amount_1 = liquidity * (su - sl)
    else:
        amount_0 = liquidity * ((1.0 / s) - (1.0 / su))
        amount_1 = liquidity * (s - sl)

    return amount_0, amount_1

In [ ]:
V3_LPS[0]

{'token_id': 221424,
 'token0': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',
 'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
 'fee_tier': 500,
 'liquidity': 33516997100150,
 'tick_range': [186440, 186640],
 'fee0': 0,
 'fee1': 0}

In [ ]:
def get_nearest_valid_tick(inp_current_tick, inp_tickspacing):
  return (inp_current_tick // inp_tickspacing) * inp_tickspacing

In [ ]:
router_address=w3.to_checksum_address('0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E')

In [ ]:
if 1>0:
  pool_addr=factory.functions.getPool(V3_LPS[0]['token0'],V3_LPS[0]['token1'],V3_LPS[0]['fee_tier']).call()
  pool_abi=fetch_abi(pool_addr,etherscan_apikey,CHAIN_ID)
  poolv_contract=w3.eth.contract(address=pool_addr, abi=pool_abi)
  pool_info=pool_current_info(poolv_contract)
  token0_address=pool_info['token0_address']
  token1_address=pool_info['token1_address']
  erc20_abi=fetch_abi("0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14",etherscan_apikey,CHAIN_ID)
  token0_contract=w3.eth.contract(address=token0_address, abi=erc20_abi)
  token1_contract=w3.eth.contract(address=token1_address, abi=erc20_abi)
  decimal0=token0_contract.functions.decimals().call()
  decimal1=token1_contract.functions.decimals().call()
  ctb=check_token_balance([pool_info['token0_address'],pool_info['token1_address']],my_wallet_address)
  print(ctb)
  current_tick=pool_info['current_tick']
  tick_spacing=pool_info['tick_spacing']
  #print(current_tick,tick_spacing)
  #print(get_nearest_valid_tick(current_tick,tick_spacing))
  a=get_nearest_valid_tick(current_tick,tick_spacing)
  b=a+(1*tick_spacing)
  #print(a,b)
  p=calculate_price_by_tick(current_tick,decimal0,decimal1)
  lowertick=0
  uppertick=0
  st=(b-a)/4
  #print(a+3*st)
  #print(a+1*st)

  if current_tick>a+st*3:
    lowertick=a
    uppertick=b+(1*tick_spacing)

  elif current_tick<(a+1*st):
    lowertick=a-(1*tick_spacing)
    uppertick=b

  else:
    pass
  print(lowertick,uppertick)
  price_lower=calculate_price_by_tick(lowertick,decimal0,decimal1)
  price_upper=calculate_price_by_tick(uppertick,decimal0,decimal1)
  res=calculate_required_amount_and_swap(p,price_lower,price_upper,ctb[0][1],ctb[1][1])
  #print(res)
  print(res['amount0_target'],res['amount1_target'])
  print(res['amount0_target']-ctb[0][1])
  print(res['amount1_target']-ctb[1][1])
  if res['amount0_target']-ctb[0][1]<0 and res['amount1_target']-ctb[1][1]>0:

    hx=make_swap(token0_address,token1_address,np.abs(res['amount0_target']-ctb[0][1]<0),
              my_wallet_address,pool_addr,router_address,'send_raw',PRIVATE_KEY)

  elif res['amount0_target']-ctb[0][1]>0 and res['amount1_target']-ctb[1][1]<0:

    hx=make_swap(token1_address,token0_address,np.abs(res['amount1_target']-ctb[1][1]),
              my_wallet_address,pool_addr,router_address,'send_raw',PRIVATE_KEY)
  print(hx)
  print(check_token_balance([pool_info['token0_address'],pool_info['token1_address']],my_wallet_address))


[['USDC', 66.633276], ['WETH', 0.05631403129150014], ['ETH', Decimal('0.209470317232092691')]]
187140 187160
267.6400943852282 0.029362575605256496
201.00681838522823
-0.02695145568624364
0x0xce96b59c80796ed62a68b7d5b0dab616cf6900bf41c1d55b3a4fbee455ad3fd4
[['USDC', 267.493953], ['WETH', 0.0293625756052565], ['ETH', Decimal('0.209470197504254602')]]


In [ ]:
if 1>0:
  pool_addr=factory.functions.getPool(V3_LPS[0]['token0'],V3_LPS[0]['token1'],V3_LPS[0]['fee_tier']).call()
  pool_abi=fetch_abi(pool_addr,etherscan_apikey,CHAIN_ID)
  poolv_contract=w3.eth.contract(address=pool_addr, abi=pool_abi)
  pool_info=pool_current_info(poolv_contract)
  token0_address=pool_info['token0_address']
  token1_address=pool_info['token1_address']
  erc20_abi=fetch_abi("0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14",etherscan_apikey,CHAIN_ID)
  token0_contract=w3.eth.contract(address=token0_address, abi=erc20_abi)
  token1_contract=w3.eth.contract(address=token1_address, abi=erc20_abi)
  decimal0=token0_contract.functions.decimals().call()
  decimal1=token1_contract.functions.decimals().call()
  ctb=check_token_balance([pool_info['token0_address'],pool_info['token1_address']],my_wallet_address)
  print(ctb)
  current_tick=pool_info['current_tick']
  tick_spacing=pool_info['tick_spacing']
  #print(current_tick,tick_spacing)
  #print(get_nearest_valid_tick(current_tick,tick_spacing))
  a=get_nearest_valid_tick(current_tick,tick_spacing)
  b=a+(1*tick_spacing)
  #print(a,b)
  p=calculate_price_by_tick(current_tick,decimal0,decimal1)
  lowertick=0
  uppertick=0
  st=(b-a)/4
  #print(a+3*st)
  #print(a+1*st)

  if current_tick>a+st*3:
    lowertick=a
    uppertick=b+(1*tick_spacing)

  elif current_tick<(a+1*st):
    lowertick=a-(1*tick_spacing)
    uppertick=b

  else:
    pass
  print(lowertick,uppertick)
  price_lower=calculate_price_by_tick(lowertick,decimal0,decimal1)
  price_upper=calculate_price_by_tick(uppertick,decimal0,decimal1)
  res=calculate_required_amount_and_swap(p,price_lower,price_upper,ctb[0][1],ctb[1][1])
  print(res)
  if res['amount0_target']-ctb[0][1]<0 and res['amount1_target']-ctb[1][1]>0:

    hx=make_swap(token0_address,token1_address,np.abs(res['amount0_target']-ctb[0][1]),
              my_wallet_address,pool_addr,router_address,'send_raw',PRIVATE_KEY)

  elif res['amount0_target']-ctb[0][1]>0 and res['amount1_target']-ctb[1][1]<0:

    hx=make_swap(token1_address,token0_address,np.abs(res['amount1_target']-ctb[1][1]),
              my_wallet_address,pool_addr,router_address,'send_raw',PRIVATE_KEY)
  print(hx)
  print(check_token_balance([pool_info['token0_address'],pool_info['token1_address']],my_wallet_address))

[['USDC', 266.493953], ['WETH', 0.029499344557920484], ['ETH', Decimal('0.209470029669631709')]]
187340 187360
{'status': 'Success', 'final_liquidity_L': 5640.912358541089, 'ratio_required': 0.00020522724264667656, 'amount0_target': 192.84447050819458, 'amount1_target': 0.039576938942055113, 'swap_details': ' 73.64948249sell token0 to buy token1 ', 'swap_amount_token0': 73.6494824918054}
insufficient allowance 
approve done
0xcdd3a8ba905832949bb13728ac38b279d995398bb1794bce036ffcf595e36467
[['USDC', 192.844471], ['WETH', 0.03957154102170548], ['ETH', Decimal('0.209469852088978049')]]


In [ ]:
def send_signed_transaction(w3: Web3, signed_tx, wait_for_receipt=True) -> str:
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    tx_hash_hex = w3.to_hex(tx_hash)
    if wait_for_receipt:
        w3.eth.wait_for_transaction_receipt(tx_hash)
    return tx_hash_hex

def handle_approve(w3: Web3, input_token_contract, swap_router_address, amount_in_wei, inp_wallet, inp_private_key, nonce) -> int:
    current_allowance = input_token_contract.functions.allowance(inp_wallet, swap_router_address).call()

    if current_allowance < amount_in_wei:
        print(f"insufficient allowance ")

        # Get EIP-1559 gas parameters
        latest_block = w3.eth.get_block('latest')
        base_fee_per_gas = latest_block['baseFeePerGas']
        max_priority_fee_per_gas = w3.eth.max_priority_fee
        max_fee_per_gas = base_fee_per_gas + 2 * max_priority_fee_per_gas # A common heuristic

        approve_tx = input_token_contract.functions.approve(
            swap_router_address,
            amount_in_wei
        ).build_transaction({
            "from": inp_wallet,
            "nonce": nonce,
            "gas": 80000,
            "maxFeePerGas": max_fee_per_gas,
            "maxPriorityFeePerGas": max_priority_fee_per_gas
        })

        signed_approve = w3.eth.account.sign_transaction(approve_tx, inp_private_key)
        send_signed_transaction(w3, signed_approve)
        print("approve done")
        return nonce + 1

    return nonce

def make_swap(input_token_address, output_token_address,
              amount_input: float, inp_wallet, inp_pool_address,
              swap_router_address, mode: str, inp_private_key):

    nonce = w3.eth.get_transaction_count(inp_wallet)

    input_token_address = w3.to_checksum_address(input_token_address)
    output_token_address = w3.to_checksum_address(output_token_address)
    pool_address = w3.to_checksum_address(inp_pool_address)
    swap_router_address = w3.to_checksum_address(swap_router_address)

    erc20_abi = fetch_abi("0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14", etherscan_apikey, CHAIN_ID)
    input_token_contract = w3.eth.contract(address=input_token_address, abi=erc20_abi)

    swap_router_abi = fetch_abi(swap_router_address, etherscan_apikey, CHAIN_ID)
    swap_router_contract = w3.eth.contract(address=swap_router_address, abi=swap_router_abi)

    pool_abi = fetch_abi(pool_address, etherscan_apikey, CHAIN_ID)
    pool_contract = w3.eth.contract(address=pool_address, abi=pool_abi)
    pool_info = pool_current_info(pool_contract)

    token_decimals = input_token_contract.functions.decimals().call()
    amount_in_wei = int(amount_input * (10**token_decimals))

    if not ((input_token_address == pool_info['token0_address'] and output_token_address == pool_info['token1_address']) or
            (input_token_address == pool_info['token1_address'] and output_token_address == pool_info['token0_address'])):
        print("Error: Input or output token address does not match the pool's tokens.")
        return False

    nonce = handle_approve(w3, input_token_contract, swap_router_address, amount_in_wei, inp_wallet, inp_private_key, nonce)

    params = {
        "tokenIn": input_token_address,
        "tokenOut": output_token_address,
        "fee": pool_info['fee_tier'],
        "recipient": inp_wallet,
        "amountIn": amount_in_wei,
        "amountOutMinimum": 0,
        "sqrtPriceLimitX96": 0
    }

    # Get EIP-1559 gas parameters for swap
    latest_block = w3.eth.get_block('latest')
    base_fee_per_gas = latest_block['baseFeePerGas']
    max_priority_fee_per_gas = w3.eth.max_priority_fee
    max_fee_per_gas = base_fee_per_gas + 2 * max_priority_fee_per_gas

    swap_tx_estimate = swap_router_contract.functions.exactInputSingle(params).build_transaction({
        "from": inp_wallet,
        "maxFeePerGas": max_fee_per_gas,
        "maxPriorityFeePerGas": max_priority_fee_per_gas
    })

    try:
        est_gas_need = w3.eth.estimate_gas(swap_tx_estimate)
    except exceptions.ContractLogicError as e:
        print(f"Error: Transaction would revert based on gas estimation. Contract Error: {e}")
        return False

    if mode == 'simulate':
        return est_gas_need

    elif mode == 'send_raw':
        swap_tx_final = swap_router_contract.functions.exactInputSingle(params).build_transaction({
            "from": inp_wallet,
            "nonce": nonce,
            "gas": est_gas_need,
            "maxFeePerGas": max_fee_per_gas,
            "maxPriorityFeePerGas": max_priority_fee_per_gas
        })

        signed_tx = w3.eth.account.sign_transaction(swap_tx_final, inp_private_key)
        tx_hash_hex = send_signed_transaction(w3, signed_tx)

        return tx_hash_hex

    else:
        print("Error: Invalid 'mode'. Use 'send_raw' or 'simulate'.")
        return False

In [ ]:
router_address=w3.to_checksum_address('0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E')

In [ ]:
make_swap('0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14','0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',0.001,my_wallet_address,'0x3289680dD4d6C10bb19b899729cda5eEF58AEfF1',router_address,'send_raw',PRIVATE_KEY)

'0x9d8b1fe5fe2ba79699fad3394574fa6c067f322c65c688ea7ae287b2de3c57c3'

In [ ]:
router_address=w3.to_checksum_address('0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E')
collect_estimated_gas=0
burn_estimated_gas=0
decrease_estimated_gas=0
mint_estimated_gas=0
sWAP_estimated_gas=0

def make_swap(input_token_address,output_token_address,amount_input:float,inp_wallet,inp_pool_address,swap_router_address,mode:str,inp_private_key):
  input_token_address=w3.to_checksum_address(input_token_address)
  erc20_abi=fetch_abi("0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14",etherscan_apikey,CHAIN_ID)
  input_token_contract=w3.eth.contract(address=input_token_address, abi=erc20_abi)
  output_token_address=w3.to_checksum_address(output_token_address)
  pool_address=w3.to_checksum_address(inp_pool_address)
  pool_abi=fetch_abi(pool_address,etherscan_apikey,CHAIN_ID)
  pool_contract=w3.eth.contract(address=pool_address, abi=pool_abi)
  swap_router_address=w3.to_checksum_address(swap_router_address)
  swap_router_abi=fetch_abi(swap_router_address,etherscan_apikey,CHAIN_ID)
  swap_router_contract=w3.eth.contract(address=swap_router_address, abi=swap_router_abi)
  pool_info=pool_current_info(pool_contract)
  amount_in_wei=int(amount_input*(10**input_token_contract.functions.decimals().call())) # Cast to int

  # Corrected the condition to call .values() and made it more robust
  if (input_token_address == pool_info['token0_address'] and output_token_address == pool_info['token1_address']) or \
     (input_token_address == pool_info['token1_address'] and output_token_address == pool_info['token0_address']):
    params = {
        "tokenIn": input_token_address,
        "tokenOut": output_token_address,
        "fee": pool_info['fee_tier'],
        "recipient": inp_wallet,
        "amountIn": amount_in_wei,
        "amountOutMinimum": 0,
        "sqrtPriceLimitX96": 0}
    nonce = w3.eth.get_transaction_count(inp_wallet)

    swap_tx = swap_router_contract.functions.exactInputSingle(params).build_transaction({
      "from": inp_wallet,
      "gasPrice": w3.eth.gas_price})

    try:
      est_gas_need=w3.eth.estimate_gas(swap_tx)
    except exceptions.ContractLogicError as e:
      print(f"Warning: Could not estimate gas for swap. Transaction would revert. Using a default gas value. Error: {e}")
      est_gas_need = 300000 # A reasonable default for a complex swap operation

    if mode=='send_raw':

      swap_tx = swap_router_contract.functions.exactInputSingle(params).build_transaction({
        "from": inp_wallet,
        "nonce": nonce,
        "gas": est_gas_need,
        "gasPrice": w3.eth.gas_price})
      signed_tx = w3.eth.account.sign_transaction(swap_tx, inp_private_key)
      tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
      return w3.to_hex(tx_hash)

    elif mode=='simulate':
      return est_gas_need
    else:
      return False
  else:
    # Handle the case where tokens do not match the pool tokens
    print("Error: Input or output token address does not match the pool's tokens.")
    return False

for lp_nft in V3_LPS:
  token0_address=lp_nft['token0']
  token1_address=lp_nft['token1']
  fee_tier=lp_nft['fee_tier']
  erc20_abi=fetch_abi("0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14",etherscan_apikey,CHAIN_ID)
  token0_contract=w3.eth.contract(address=token0_address, abi=erc20_abi)
  token1_contract=w3.eth.contract(address=token1_address, abi=erc20_abi)
  decimal0=token0_contract.functions.decimals().call()
  decimal1=token1_contract.functions.decimals().call()
  pool_address_v=factory.functions.getPool(token0_address, token1_address, fee_tier).call()
  pool_abi=fetch_abi(pool_address_v,etherscan_apikey,CHAIN_ID)
  pool_contract=w3.eth.contract(address=pool_address_v, abi=pool_abi)
  pool_info=pool_current_info(pool_contract)
  # Corrected calculation for fee_balance_in_eth to handle token decimals
  fee0_price = calculate_price_from_sqrt_price(pool_info['sqrtprice'], decimal0, decimal1)
  fee_balance_in_eth = (lp_nft['fee0'] / (10**decimal0)) * fee0_price + (lp_nft['fee1'] / (10**decimal1))
  POSITION_TOKEN_ID = lp_nft['token_id']
  liquidity_amount = lp_nft['liquidity']

  # A. Decrease Liquidity
  params_decrease = {
            "tokenId": POSITION_TOKEN_ID,
            "liquidity": liquidity_amount,
            "amount0Min": 0,
            "amount1Min": 0,
            "deadline": int(time.time()) + 1200}

  decrease_tx = position_manager_contract.functions.decreaseLiquidity(params_decrease).build_transaction({
            "from": my_wallet_address,
            "gasPrice": w3.eth.gas_price})

  decrease_estimated_gas = w3.eth.estimate_gas(decrease_tx)

  # B. Collect Fees/Tokens
  MAX_UINT128 = 2**128 - 1
  params_collect = {
            "tokenId": POSITION_TOKEN_ID,
            "recipient": my_wallet_address,
            "amount0Max": MAX_UINT128,
            "amount1Max": MAX_UINT128}

  collect_tx = position_manager_contract.functions.collect(params_collect).build_transaction({
            "from": my_wallet_address,
            "gasPrice": w3.eth.gas_price})

  # This estimate should be for collect_tx, not decrease_tx
  collect_estimated_gas = w3.eth.estimate_gas(collect_tx)

  # C. Burn NFT
  # The burn transaction will revert if the position is not cleared (liquidity 0 and fees collected).
  # During the gas estimation phase, these actions haven't happened on the blockchain.
  # We will wrap this in a try-except block to prevent the crash,
  # and assign a placeholder if estimation fails.
  try:
    burn_tx = position_manager_contract.functions.burn(POSITION_TOKEN_ID).build_transaction({
              "from": my_wallet_address,
              "gasPrice": w3.eth.gas_price})
    burn_estimated_gas = w3.eth.estimate_gas(burn_tx)
  except exceptions.ContractLogicError as e:
    print(f"Warning: Could not estimate gas for burn. Position likely not cleared. Using a default gas value. Error: {e}")
    burn_estimated_gas = 100000 # A reasonable default gas estimate for a simple burn

  ctb=check_token_balance([token0_address,token1_address],my_wallet_address)
  # Ensure the make_swap function is correctly called with desired amount
  # The current ctb[0][1]/10 is arbitrary, ensure it's suitable for simulation.
  swap_estimated_gas=make_swap(token0_address,token1_address,ctb[0][1]/10,my_wallet_address,pool_address_v,router_address,'simulate',PRIVATE_KEY)

  def get_nearest_valid_tick(tick, spacing):
    return (tick // spacing) * spacing

  valid_current_tick = get_nearest_valid_tick(pool_info['current_tick'], pool_info['tick_spacing'])

  # Recalculate tick_lower and tick_upper for mint to match logic below
  # The original code has tick_lower/upper calculated twice, once with +/- 10*spacing
  # and once with +/- 1 or 2*spacing. The latter is what's used for actual minting.
  # Let's ensure consistency for mint_estimated_gas calculation.

  a=valid_current_tick - (1 * pool_info['tick_spacing'])
  b=valid_current_tick + (1 * pool_info['tick_spacing']) # This was tick_upper = valid_current_tick + (1 * pool_info['tick_spacing'])

  # Adjusted tick range for estimation to match actual minting logic in else block
  est_tick_lower = 0
  est_tick_upper = 0
  if valid_current_tick > (2*((a+b)/3)):
    est_tick_lower = valid_current_tick - (1 * pool_info['tick_spacing'])
    est_tick_upper = valid_current_tick + (2 * pool_info['tick_spacing'])
  elif valid_current_tick < (1)*((a+b)/3):
    est_tick_lower = valid_current_tick - (2 * pool_info['tick_spacing'])
    est_tick_upper = valid_current_tick + (1 * pool_info['tick_spacing'])
  else:
    est_tick_lower = valid_current_tick - (1 * pool_info['tick_spacing'])
    est_tick_upper = valid_current_tick + (1 * pool_info['tick_spacing'])


  # ----------------------------------------------
  # req_amount0 and req_amount1 used here are from check_token_balance
  # which reflects current wallet balances.
  # For minting, the actual amounts will come from L = init_amounts_liquid(...).
  # We should use a proxy for these amounts for estimation or use the L values directly.
  # Assuming ctb[0][1] and ctb[1][1] represent the available tokens for re-minting.
  est_req_amount0 = ctb[0][1]
  est_req_amount1 = ctb[1][1]

  # Need to ensure init_amounts_liquid is called with appropriate parameters for estimation
  # For now, using the estimated req_amount0 and req_amount1 directly in params.
  # This might not be fully accurate if init_amounts_liquid changes these significantly.

  # Calculate L for mint estimation
  est_price_lower=calculate_price_by_tick(est_tick_lower,decimal0,decimal1)
  est_price_upper=calculate_price_by_tick(est_tick_upper,decimal0,decimal1)
  est_eth_sepolia_price=1/calculate_price_from_sqrt_price(pool_info['sqrtprice'],decimal0,decimal1)
  est_capital=est_req_amount0*1+est_req_amount1*est_eth_sepolia_price # Hypo capital for estimation
  est_L, est_final_amount_0, est_final_amount_1 = init_amounts_liquid(
      pool_info['sqrtprice'], decimal0, decimal1, 1, est_eth_sepolia_price, est_price_lower, est_price_upper,
      inp_hypo_capital=est_capital
  )


  params_mint = {
    "token0": token0_address,
    "token1": token1_address,
    "fee": pool_info['fee_tier'],
    "tickLower": est_tick_lower,
    "tickUpper": est_tick_upper,
    "amount0Desired": int(est_final_amount_0 * (10**decimal0)),
    "amount1Desired": int(est_final_amount_1 * (10**decimal1)),
    "amount0Min": 0,
    "amount1Min": 0,
    "recipient": my_wallet_address,
    "deadline": int(time.time()) + 1200}

  # ساخت تراکنش
  tx_mint = position_manager_contract.functions.mint(params_mint).build_transaction({
    "from": my_wallet_address,
    "gasPrice": w3.eth.gas_price})

  mint_estimated_gas = w3.eth.estimate_gas(tx_mint)
  # ------------------------------------------------------------------------------
  all_gas_need=decrease_estimated_gas+collect_estimated_gas+burn_estimated_gas+mint_estimated_gas+swap_estimated_gas
  # Ensure w3.to_wei is used correctly for fee_balance_in_eth if comparing with gas costs.
  # The fee_balance_in_eth is in real units, w3.eth.gas_price is in wei.
  # So, convert fee_balance_in_eth to wei before comparison.
  # Original: w3.to_wei(fee_balance_in_eth, 'ether') > (all_gas_need * w3.eth.gas_price)
  # This assumes fee_balance_in_eth is already in ether. Let's verify.
  # fee_balance_in_eth = (lp_nft['fee0'] / (10**decimal0)) * fee0_price + (lp_nft['fee1'] / (10**decimal1))
  # This calculation makes fee_balance_in_eth a float.
  # So, w3.to_wei(fee_balance_in_eth, 'ether') is correct.

  if pool_info['current_tick']>lp_nft['tick_range'][0] and pool_info['current_tick']<lp_nft['tick_range'][1] or w3.to_wei(fee_balance_in_eth, 'ether')>(all_gas_need*w3.eth.gas_price):
    pass
  else:
    # Get the nonce for the first transaction
    current_nonce = w3.eth.get_transaction_count(my_wallet_address)

    # A. Decrease Liquidity
    params_decrease = {
            "tokenId": POSITION_TOKEN_ID,
            "liquidity": liquidity_amount,
            "amount0Min": 0,
            "amount1Min": 0,
            "deadline": int(time.time()) + 1200}

    decrease_tx = position_manager_contract.functions.decreaseLiquidity(params_decrease).build_transaction({
            "from": my_wallet_address,
            "nonce": current_nonce,
            "gas": decrease_estimated_gas, # Use the estimated gas
            "gasPrice": w3.eth.gas_price})

    signed_decrease_tx = w3.eth.account.sign_transaction(decrease_tx, PRIVATE_KEY)
    hash_decrease = w3.eth.send_raw_transaction(signed_decrease_tx.raw_transaction)
    print(f"Decrease TX SENT for ID {POSITION_TOKEN_ID}: {w3.to_hex(hash_decrease)}")

    # Wait for the transaction to be mined to update the state
    w3.eth.wait_for_transaction_receipt(hash_decrease, timeout=600)
    current_nonce += 1 # Increment nonce for the next transaction

    # B. Collect Fees/Tokens
    MAX_UINT128 = 2**128 - 1
    params_collect = {
            "tokenId": POSITION_TOKEN_ID,
            "recipient": my_wallet_address,
            "amount0Max": MAX_UINT128,
            "amount1Max": MAX_UINT128}

    collect_tx = position_manager_contract.functions.collect(params_collect).build_transaction({
            "from": my_wallet_address,
            "nonce": current_nonce,
            "gas": collect_estimated_gas, # Use the estimated gas
            "gasPrice": w3.eth.gas_price})

    signed_collect_tx = w3.eth.account.sign_transaction(collect_tx, PRIVATE_KEY)
    hash_collect = w3.eth.send_raw_transaction(signed_collect_tx.raw_transaction)
    print(f"Collect TX SENT for ID {POSITION_TOKEN_ID}: {w3.to_hex(hash_collect)}")

    # Wait for the transaction to be mined to update the state
    w3.eth.wait_for_transaction_receipt(hash_collect, timeout=600)
    current_nonce += 1 # Increment nonce for the next transaction

    # C. Burn NFT
    burn_tx_active = position_manager_contract.functions.burn(POSITION_TOKEN_ID).build_transaction({
            "from": my_wallet_address,
            "nonce": current_nonce,
            "gas": burn_estimated_gas, # Use the estimated gas
            "gasPrice": w3.eth.gas_price})
    signed_burn_tx_active = w3.eth.account.sign_transaction(burn_tx_active, PRIVATE_KEY)
    hash_burn_active = w3.eth.send_raw_transaction(signed_burn_tx_active.raw_transaction)
    print(f"Burn TX SENT for ID {POSITION_TOKEN_ID}: {w3.to_hex(hash_burn_active)}")

    # Wait for the transaction to be mined to update the state
    w3.eth.wait_for_transaction_receipt(hash_burn_active, timeout=600)
    current_nonce += 1 # Increment nonce for the next transaction


    ctb=check_token_balance([token0_address,token1_address],my_wallet_address)
    # The swap part
    # If a swap is needed, this would be an additional transaction.
    # The original code just estimated it, but didn't send it.
    # If it is to be executed, it should be sent here with a separate nonce.


    def get_nearest_valid_tick(tick, spacing):
      return (tick // spacing) * spacing

    valid_current_tick = get_nearest_valid_tick(pool_info['current_tick'], pool_info['tick_spacing'])

    a=valid_current_tick - (1 * pool_info['tick_spacing'])
    b=valid_current_tick + (1 * pool_info['tick_spacing']) # Fixed for consistency

    tick_lower = 0
    tick_upper = 0
    if valid_current_tick>(2*((a+b)/3)):
      tick_lower = valid_current_tick - (1 * pool_info['tick_spacing'])
      tick_upper = valid_current_tick + (2 * pool_info['tick_spacing'])
    elif valid_current_tick<(1)*((a+b)/3):
      tick_lower = valid_current_tick - (2 * pool_info['tick_spacing'])
      tick_upper = valid_current_tick + (1 * pool_info['tick_spacing'])
    else:
      tick_lower = valid_current_tick - (1 * pool_info['tick_spacing'])
      tick_upper = valid_current_tick + (1 * pool_info['tick_spacing'])

    # ----------------------------------------------
    req_amount0=ctb[0][1] # Current balance of token0 after previous operations
    req_amount1=ctb[1][1] # Current balance of token1 after previous operations

    price_lower=calculate_price_by_tick(tick_lower,decimal0,decimal1)
    price_upper=calculate_price_by_tick(tick_upper,decimal0,decimal1)
    eth_sepolia_price=1/calculate_price_from_sqrt_price(pool_info['sqrtprice'],decimal0,decimal1)
    capital=req_amount0*1+req_amount1*eth_sepolia_price # Total capital in terms of token0 or a common base

    L, final_amount_0, final_amount_1 = init_amounts_liquid(
        pool_info['sqrtprice'], decimal0, decimal1, 1, eth_sepolia_price, price_lower, price_upper,
        inp_hypo_capital=capital # Use calculated capital for minting
    )

    params_mint_final = {
      "token0": token0_address,
      "token1": token1_address,
      "fee": pool_info['fee_tier'],
      "tickLower": tick_lower,
      "tickUpper": tick_upper,
      "amount0Desired": int(final_amount_0 * (10**decimal0)),
      "amount1Desired": int(final_amount_1 * (10**decimal1)),
      "amount0Min": 0,
      "amount1Min": 0,
      "recipient": my_wallet_address,
      "deadline": int(time.time()) + 1200}

    # ساخت تراکنش
    tx_mint_final = position_manager_contract.functions.mint(params_mint_final).build_transaction({
    "from": my_wallet_address,
    "nonce": current_nonce, # Use incremented nonce
    "gas": mint_estimated_gas, # Use estimated gas
    "gasPrice": w3.eth.gas_price})

    signed_tx_mint_final = w3.eth.account.sign_transaction(tx_mint_final, PRIVATE_KEY)
    tx_hash_mint_final = w3.eth.send_raw_transaction(signed_tx_mint_final.raw_transaction)

    print("Mint TX SENT:", w3.to_hex(tx_hash_mint_final))

ContractLogicError: ('execution reverted: STF', '0x08c379a0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000035354460000000000000000000000000000000000000000000000000000000000')

In [ ]:
check_liquidity_v3_nft(my_wallet_address,position_manager_contract)

تعداد NFT پوزیشن‌ها: 1


[{'token_id': 221424,
  'token0': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238',
  'token1': '0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14',
  'fee_tier': 500,
  'liquidity': 33516997100150,
  'tick_range': [186440, 186640],
  'fee0': 0,
  'fee1': 0}]